In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import re
import string
import nltk 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import tree
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.pipeline import Pipeline

%matplotlib inline 

In [2]:
df = pd.read_csv("olid-training-v1.0.csv",delimiter="\t")
df

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN
...,...,...,...,...,...
13235,95338,@USER Sometimes I get strong vibes from people...,OFF,TIN,IND
13236,67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT,NaN,NaN
13237,82921,@USER And why report this garbage. We don't g...,OFF,TIN,OTH
13238,27429,@USER Pussy,OFF,UNT,NaN


In [61]:
set(df.subtask_c)

{'GRP', 'IND', 'OTH', nan}

In [3]:
def demoji(text):
        emoji_pattern = re.compile("["
                                  u"\U0001F600-\U0001F64F"
                                  u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                  u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                  u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                  u"\U00002702-\U000027B0"
                                  u"\U000024C2-\U0001F251"
                                  u"\U00010000-\U0010ffff"
                                   "]+", flags=re.UNICODE)
        return(emoji_pattern.sub(r'', text))
                                  
     
                                  
   
 

df[u'tweet'] = df[u'tweet'].astype(str)
df[u'tweet'] = df[u'tweet'].apply(lambda x:demoji(x))
df.to_csv('output.csv',index=False, encoding='utf-8') # save to csv file


In [4]:
df

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN
...,...,...,...,...,...
13235,95338,@USER Sometimes I get strong vibes from people...,OFF,TIN,IND
13236,67210,Benidorm Creamfields Maga Not too shabb...,NOT,NaN,NaN
13237,82921,@USER And why report this garbage. We don't g...,OFF,TIN,OTH
13238,27429,@USER Pussy,OFF,UNT,NaN


In [5]:
for i in range(len(df.subtask_b)):
    if(df.subtask_a[i] == "NOT"):
               df.subtask_b[i] = "NOT"

D:\Application\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
training = df.tweet
labels = df.subtask_b

In [7]:
for i in range(len(training)):## delete all @USER
    training[i] = training[i].replace("@USER","")

D:\Application\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
training

0         She should ask a few native Americans what th...
1           Go home you’re drunk!!!  #MAGA #Trump2020  URL
2        Amazon is investigating Chinese employees who ...
3         Someone should'veTaken" this piece of shit to...
4          Obama wanted liberals &amp; illegals to move...
                               ...                        
13235     Sometimes I get strong vibes from people and ...
13236    Benidorm   Creamfields   Maga    Not too shabb...
13237     And why report this garbage.  We don't give a...
13238                                                Pussy
13239    #Spanishrevenge vs. #justice #HumanRights and ...
Name: tweet, Length: 13240, dtype: object

In [9]:
for i in range(len(training)):## lowercase
    training[i] = training[i].lower()

D:\Application\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text
training = training.apply(lambda x: remove_punct(x))

In [11]:
for i in range(len(training)):## lowercase
    training[i] = training[i].replace("url","")

In [12]:
def tokenization(text):###split all words
    text = re.split('\W+', text)
    return text

training_token = training.apply(lambda x: tokenization(x.lower()))


In [13]:
training_token

0        [, she, should, ask, a, few, native, americans...
1              [, go, home, you, re, drunk, maga, trump, ]
2        [amazon, is, investigating, chinese, employees...
3        [, someone, shouldvetaken, this, piece, of, sh...
4        [, obama, wanted, liberals, amp, illegals, to,...
                               ...                        
13235    [, sometimes, i, get, strong, vibes, from, peo...
13236    [benidorm, creamfields, maga, not, too, shabby...
13237    [, and, why, report, this, garbage, we, dont, ...
13238                                            [, pussy]
13239    [spanishrevenge, vs, justice, humanrights, and...
Name: tweet, Length: 13240, dtype: object

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LeoFishLiao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
stopword = set(stopwords.words('english'))

In [16]:
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
training_stop = training.apply(lambda x: remove_stopwords(x))


In [17]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [word for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text

In [18]:
countVectorizer = CountVectorizer(analyzer=clean_text)
countVector = countVectorizer.fit_transform(training)

In [19]:
df.tweet = training


In [20]:
training = df.tweet[df.subtask_b!="NOT"]
labels = df.subtask_b[df.subtask_b!="NOT"]

In [21]:
print('{} Number of tweets has {} words'.format(countVector.shape[0], countVector.shape[1]))

13240 Number of tweets has 19925 words


In [22]:
count_vect_df = pd.DataFrame(countVector.toarray(), columns=countVectorizer.get_feature_names())
# count_vect_df.to_csv('vector.csv')

In [23]:
text_clf = Pipeline([
        ('conuntVectorizer', CountVectorizer(analyzer=clean_text,ngram_range=(1,2))),
        ('clf', MultinomialNB()),])

In [24]:
text_clf.fit(training,labels)

Pipeline(memory=None,
         steps=[('conuntVectorizer',
                 CountVectorizer(analyzer=<function clean_text at 0x00000267D98A7948>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [25]:
from sklearn.model_selection import cross_val_score
# clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(text_clf, training, labels, cv=5)

In [26]:
scores

array([0.88081725, 0.88068182, 0.87840909, 0.87954545, 0.87827076])

In [27]:
# print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [28]:
test = pd.read_csv('testset-levelb.csv',delimiter = "\t").tweet

In [29]:
predicted = text_clf.predict(test)

In [30]:
test_label = pd.read_csv('labels-levelb.csv')

In [31]:
test_label = test_label.Label

In [32]:
np.mean(predicted == test_label)

0.875

In [33]:
#使用其他分类器-------支持向量机

In [34]:
from sklearn.svm import LinearSVC

In [35]:
text_clf2 = Pipeline([
        ('conuntVectorizer', CountVectorizer(analyzer=clean_text,ngram_range=(2,4))),
        ('tfidf', TfidfTransformer()),
        ('clf', LinearSVC())])

In [36]:
text_clf2.fit(training,labels)

Pipeline(memory=None,
         steps=[('conuntVectorizer',
                 CountVectorizer(analyzer=<function clean_text at 0x00000267D98A7948>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(2, 4), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept

In [62]:
scores = cross_val_score(text_clf, training, labels, cv=5)
scores

array([0.88081725, 0.88068182, 0.87840909, 0.87954545, 0.87827076])

In [63]:
predicted = text_clf2.predict(test)

In [64]:
np.mean(predicted == test_label)

0.8958333333333334

In [65]:
pre = pd.Series(predicted,name="predicted")
lab = pd.Series(test_label)
pd.DataFrame([test,lab,pre]).T.loc[200:210]

,tweet,Label,predicted
200,#ICYMI - Liberals' Favorite Myth That Obama Wa...,TIN,TIN
201,@USER To demonstrate how wholly incapable of c...,TIN,TIN
202,And she has a pet??? Fucking disgusting URL,TIN,UNT
203,#LiberalHypocrisy #TacoBell When Liberals ask ...,TIN,TIN
204,#BlackInkCrewCHI lily they don’t hate you....y...,TIN,TIN
205,#JusticeForSoniasFather Mr Usman buzdar sb! p...,TIN,TIN
206,@USER - WORST POTUS. EVER!,TIN,TIN
207,#Conservatives have no genetic ability to unde...,TIN,TIN
208,#OITNB. She is the worst public defender. Trai...,UNT,TIN
209,#TheView #BothamJean I don't believe #AmyGuyge...,TIN,TIN


In [40]:
##使用随机森林

In [41]:
from sklearn.ensemble import RandomForestClassifier

In [42]:
text_clf3 = Pipeline([
        ('conuntVectorizer', CountVectorizer(analyzer=clean_text,ngram_range=(2,2))),
        ('transform',TfidfTransformer()),
        ('clf',RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0))])

In [43]:
text_clf3.fit(training,labels)

Pipeline(memory=None,
         steps=[('conuntVectorizer',
                 CountVectorizer(analyzer=<function clean_text at 0x00000267D98A7948>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(2, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?...
                ('clf',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=3,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
         

In [44]:
scores = cross_val_score(text_clf, training, labels, cv=5)
scores

array([0.88081725, 0.88068182, 0.87840909, 0.87954545, 0.87827076])

In [45]:
predicted = text_clf3.predict(test)

In [46]:
np.mean(predicted == test_label)

0.8875

In [47]:
#使用Xgboost

In [48]:
!pip install xgboost

In [49]:
import xgboost as xgb

In [50]:
text_clf4 = Pipeline([
        ('conuntVectorizer', CountVectorizer(analyzer=clean_text,ngram_range=(2,4))),
        ('transform',TfidfTransformer()),
        ('clf',xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1))])

In [51]:
text_clf4.fit(training,labels)

Pipeline(memory=None,
         steps=[('conuntVectorizer',
                 CountVectorizer(analyzer=<function clean_text at 0x00000267D98A7948>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(2, 4), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?...
                               interaction_constraints=None, learning_rate=0.1,
                               max_delta_step=0, max_depth=7,
                               min_child_weight=1, missing=nan,
                               monotone_constraints=None, n_estimators=200,
                               n_jobs=10, nthread=10, num_parallel_tree=1,
           

In [52]:
scores = cross_val_score(text_clf, training, labels, cv=5)
scores

array([0.88081725, 0.88068182, 0.87840909, 0.87954545, 0.87827076])

In [53]:
predicted = text_clf4.predict(test)

In [54]:
np.mean(predicted == test_label)

0.8916666666666667

In [55]:
##使用SDG

In [56]:
text_clf5 = Pipeline([
        ('conuntVectorizer', CountVectorizer(analyzer=clean_text)),
        ('tfidf', TfidfTransformer()),
        ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None))])

In [57]:
text_clf5.fit(training,labels)

Pipeline(memory=None,
         steps=[('conuntVectorizer',
                 CountVectorizer(analyzer=<function clean_text at 0x00000267D98A7948>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?...
                ('clf',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                               max_iter=5, n_iter_no_c

In [58]:
predicted = text_clf5.predict(test)

In [59]:
np.mean(predicted == test_label)

0.8875